## DGGS API dev, H3 zoning and querying

In [3]:
import yaml

**Version 0.0.6 process style superceded**

In [11]:
fn = "../dggs_api_server/swagger/swagger-0.0.6.yaml"

In [12]:
with open(fn) as fh:
    read_data = yaml.load(fh, Loader=yaml.FullLoader)

# Print YAML data before sorting
# print(read_data)

In [13]:
reorder = {}

for k, v in read_data['paths'].items():
    # print(k)
    for method in ['get', 'post', 'options', 'put', 'delete', 'head']:
        # print(v['x-openapi-router-controller'])
        if method in v.keys():
            path = v[method]['x-openapi-router-controller']
            print(f"{k} [{method}]: {path}")
            reorder.update( { f"{k} [{method}]": path} )

/ [get]: dggs_api_server.controllers.capabilities_controller
/conformance [get]: dggs_api_server.controllers.capabilities_controller
/collections [get]: dggs_api_server.controllers.capabilities_controller
/collections/{collectionId}/describe [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zones [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone [get]: dggs_api_server.controllers.dggs_access_controller
/collections/{collectionId}/zone/{zoneId}/distance [get]: dggs_api_server.controllers.zone_query_ops_controller
/collections/{collectionId}/zone/{zoneId}/buffer [get]: dggs_api_server.controllers.zone_query_set_controller
/collections/{collectionId}/zone/{zoneId}/difference [get]: dggs_api_server.controllers.zone_query_set_controller
/collections/{collectionId}/zone/{zoneId}/intersection [get]: dggs_api_server.controllers.zone_query_set_controller
/collections/{collectionId}/zone/{zoneId}/symDifference [get]: dg

**Version 0.0.6a current state ogc api dggs**

In [16]:
fn2 = "../dggs_api_server/swagger/swagger-0.0.6a.yaml"

In [19]:
with open(fn2, encoding="utf-8") as fh:
    read_data2 = yaml.load(fh, Loader=yaml.FullLoader)

In [20]:
reorder2 = {}

for k, v in read_data2['paths'].items():
    # print(k)
    for method in ['get', 'post', 'options', 'put', 'delete', 'head']:
        # print(v['x-openapi-router-controller'])
        if method in v.keys():
            path = v[method]['x-openapi-router-controller'].replace("<<","").replace(">>", "_")
            print(f"{k} [{method}]: {path}")
            reorder2.update( { f"{k} [{method}]": path} )

/ [get]: dggs_api_server.controllers.ogc_api_common_controller
/api [get]: dggs_api_server.controllers.ogc_api_common_controller
/conformance [get]: dggs_api_server.controllers.ogc_api_common_controller
/collections [get]: dggs_api_server.controllers.ogc_api_common_controller
/collections/{collectionId} [get]: dggs_api_server.controllers.ogc_api_common_controller
/dggs [get]: dggs_api_server.controllers.dggs_core_controller
/dggs/{dggsRSID} [get]: dggs_api_server.controllers.dggs_core_controller
/dggs/{dggsRSID}/zones [get]: dggs_api_server.controllers.dggs_core_controller
/dggs/{dggsRSID}/zone/{ZonalID} [get]: dggs_api_server.controllers.dggs_core_controller
/dggs/{dggsRSID}/zone/{ZonalID}/distance [get]: dggs_api_server.controllers.dggs_zone_query_operations_controller
/collections/{collectionId}/dggs/{dggsRSID}/zone/{ZonalID}/distance [get]: dggs_api_server.controllers.collection_zone_query_operations_controller
/dggs/{dggsRSID}/zone/{ZonalID}/buffer [get]: dggs_api_server.controlle

In [25]:
import requests

In [26]:
# API_BASE="https://dggs-api-bozea3cspa-ew.a.run.app/dggs-api"
API_BASE="http://localhost:8080/dggs-api"

In [63]:
# / [get]
r = requests.get(API_BASE)
r.json()

{'links': {'description': 'Access to data about Estonia via a Web API that aims to conform to the OGC API DGGS specification.',
  'links': [{'href': 'http://localhost:8080/dggs-api',
    'rel': 'self',
    'title': 'this document',
    'type': 'application/json'},
   {'href': 'http://localhost:8080/dggs-api/openapi.json',
    'rel': 'service-desc',
    'title': 'the API definition',
    'type': 'application/vnd.oai.openapi+json;version=3.0'},
   {'href': 'http://localhost:8080/dggs-api/ui',
    'rel': 'service-doc',
    'title': 'the API documentation',
    'type': 'text/html'},
   {'href': 'http://localhost:8080/dggs-api/conformance',
    'rel': 'conformance',
    'title': 'OGC API conformance classes implemented by this server',
    'type': 'application/json'},
   {'href': 'http://localhost:8080/dggs-api/collections',
    'rel': 'data',
    'title': 'Information about the feature collections',
    'type': 'application/json'}],
  'title': 'DGGS coded data in Estonia'}}

In [64]:
# /collections [get]
r = requests.get(API_BASE + "/collections")
r.json()

{'collections': [{'attribution': 'attribution',
   'crs': 'H3',
   'description': '30m SRTM sampled at res 8 covering Estonia',
   'extent': {'bbox': [24.0, 58.0, 27.0, 59.0], 'crs': 'WGS84'},
   'id': 'srtm_30m_estonia_h3',
   'keywords': [{'keyword': 'elevation'}],
   'links': [{'href': 'https://landscape-geoinformatics.ut.ee/dggs-en',
     'rel': 'alternate',
     'title': '30m SRTM sampled at res 8 covering Estonia',
     'type': 'text/html'}],
   'title': 'srtm_30m_estonia_h3'},
  {'attribution': 'attribution',
   'crs': 'RHEALPIX',
   'description': '30m SRTM re-sampled at res 8 from the H3 dataset covering Estonia',
   'extent': {'bbox': [24.0, 58.0, 27.0, 59.0], 'crs': 'WGS84'},
   'id': 'srtm_30m_estonia_rhpix',
   'keywords': [{'keyword': 'elevation'}],
   'links': [{'href': 'https://landscape-geoinformatics.ut.ee/dggs-en',
     'rel': 'alternate',
     'title': '30m SRTM re-sampled at res 8 from the H3 dataset covering Estonia',
     'type': 'text/html'}],
   'title': 'srtm_

In [60]:
collection = r.json()['collections'][0]
collectionId = collection['id']
print(f"working with <{collectionId}>")

working with <srtm_30m_estonia_h3>


In [62]:
# /collections/{collectionId}/describe [get]
r = requests.get(API_BASE + f"/collections/{collectionId}")
r.json()

{'attribution': 'attribution',
 'crs': 'H3',
 'description': '30m SRTM sampled at res 8 covering Estonia',
 'extent': {'bbox': [24.0, 58.0, 27.0, 59.0], 'crs': 'WGS84'},
 'id': 'srtm_30m_estonia_h3',
 'keywords': [{'keyword': 'elevation'}],
 'links': [{'href': 'https://landscape-geoinformatics.ut.ee/dggs-en',
   'rel': 'alternate',
   'title': '30m SRTM sampled at res 8 covering Estonia',
   'type': 'text/html'}],
 'title': 'srtm_30m_estonia_h3'}

In [7]:
import folium
from h3 import h3

In [8]:
def visualize_polygon_features(features, color="red", folium_map=None):
    geom_ext = features[0]['geometry']['coordinates'][0]
    lng = [p[0] for p in geom_ext]
    lat = [p[1] for p in geom_ext]
    if folium_map is None:
        m = folium.Map(height=600, width=800, location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    
    style1 = {'color': '#228B22'}
    style2 = {'color': '#00FFFFFF'}
    
    for f in features:
        # folium.GeoJson(f, name=features[0]['id'],style_function=lambda x: style1 if color == 'red' else style2  ).add_to(m)
        folium.GeoJson(f, name=features[0]['id'],style_function=lambda x: { 'color': color }  ).add_to(m)
    return m

In [9]:
import copy

def dggs_json_to_geojson_feature(dggs_json):
    cell_id = dggs_json['id']
    cell_id2 = dggs_json['geometry'][0]
    assert(cell_id == cell_id2)
    geom = h3.h3_to_geo_boundary(cell_id, geo_json=True)
    geojson_feature = dggs_json.copy()
    geojson_feature['geometry'] = {'type': 'Polygon',
            'coordinates': [
                geom
            ]
        }
    return geojson_feature


In [10]:
# /collections/{collectionId}/zones [get]
query_params = {'resolution': 5, 'bbox': "25.32,58.27,26.82,58.67", 'limit': 200}
r = requests.get(API_BASE + f"/collections/{collectionId}/zones", params=query_params)
zone_json = r.json()

In [11]:
len(zone_json['features'])

19

In [12]:
zone_geojson_collection = [dggs_json_to_geojson_feature(f) for f in zone_json['features']]
m1 = visualize_polygon_features(zone_geojson_collection, color="red", folium_map=None)

In [13]:
m1

In [14]:
# /collections/{collectionId}/zones [get]
query_params = {'resolution': 6, 'bbox': "25.32,58.27,26.82,58.67", 'limit': 200}
r = requests.get(API_BASE + f"/collections/{collectionId}/zones", params=query_params)
zone_json = r.json()

In [15]:
zone_geojson_collection = [dggs_json_to_geojson_feature(f) for f in zone_json['features']]
m2 = visualize_polygon_features(zone_geojson_collection, color="blue", folium_map=m1)

In [16]:
m2

## manual H3 cells, size and resolutions for reference

In [99]:
minx, miny, maxx, maxy = [25.32, 58.27, 26.82, 58.67]
resolution = 5

area_json = {'type': 'Polygon',
            'coordinates': [
                [
                    [miny, minx],
                    [miny, maxx],
                    [maxy, maxx],
                    [maxy, minx],
                    [miny, minx]
                ]
            ]
        }
zone_id_list_from_fill = h3.polyfill(area_json, resolution)

In [100]:
len(zone_id_list_from_fill)

19

In [121]:
h3.h3_get_resolution("8511346bfffffff")

5

In [115]:
"8511340bfffffff" in zone_id_list_from_fill

True

In [103]:
zone_id_list_from_fill

{'8511340bfffffff',
 '8511341bfffffff',
 '85113443fffffff',
 '85113447fffffff',
 '8511344ffffffff',
 '85113453fffffff',
 '85113457fffffff',
 '85113463fffffff',
 '8511346bfffffff',
 '8511346ffffffff',
 '85113473fffffff',
 '8511347bfffffff',
 '851134c3fffffff',
 '851134c7fffffff',
 '851134cbfffffff',
 '851134cffffffff',
 '851134dbfffffff',
 '851136a7fffffff',
 '851136b7fffffff'}

In [101]:


m = visualize_hexagons(zone_id_list_from_fill)
display(m)



## RHEALPIX test

In [17]:
import rhealpixdggs as rhpix
import rhealpixdggs.dggs as rhpix_dggs
import copy

def get_rhpix_geoid():
    rhpix_geoid = rhpix_dggs.WGS84_003
    return rhpix_geoid

default_rhpix_geoid = get_rhpix_geoid()

def rhpix_cell_id_to_suid(cell_id):
    resolution = len(cell_id)-1
    suid_text = [x for x in cell_id]
    suid_chain = [ str(suid_text[0]) ]
    for x in suid_text[1:]:
        suid_chain.append(int(x))
    return tuple(suid_chain)


def rhpix_cell_id_to_geom(cell_id, rhpix_geoid=default_rhpix_geoid):
    cell_suid = rhpix_cell_id_to_suid(cell_id)
    c = rhpix_geoid.cell(suid=cell_suid)
    vx = c.vertices(plane=False)
    vx.append(vx[0])
    return vx


def rhpix_dggs_json_to_geojson_feature(dggs_json):
    cell_id = dggs_json['id']
    cell_id2 = dggs_json['geometry'][0]
    assert(cell_id == cell_id2)
    
    geom = rhpix_cell_id_to_geom(cell_id, rhpix_geoid=default_rhpix_geoid)
    
    geojson_feature = dggs_json.copy()
    geojson_feature['geometry'] = {'type': 'Polygon',
            'coordinates': [
                geom
            ]
        }
    return geojson_feature

In [18]:
# /collections/{collectionId}/describe [get]
collectionId = "corine_lulc_estonia_rhpix"

r = requests.get(API_BASE + f"/collections/{collectionId}/describe")
r.json()

{'description': 'CORINE Landuse/land cover classes re-sampled from H3 dataset at res 8 covering Estonia',
 'dggs-id': 'RHEALPIX',
 'id': 'corine_lulc_estonia_rhpix',
 'links': ['https://landscape-geoinformatics.ut.ee/dggs-en'],
 'resolutions': ['8', '7', '6', '5', '4', '3', '2'],
 'title': 'corine_lulc_estonia_rhpix'}

In [26]:
# /collections/{collectionId}/zones [get]
query_params = {'resolution': 7, 'bbox': "25.32,58.27,26.82,58.67", 'limit': 500}
r = requests.get(API_BASE + f"/collections/{collectionId}/zones", params=query_params)
zone_json = r.json()

In [ ]:
zone_json

In [28]:
zone_geojson_collection = [rhpix_dggs_json_to_geojson_feature(f) for f in zone_json['features']]
m3 = visualize_polygon_features(zone_geojson_collection, color="green", folium_map=m2)

In [29]:
m3